In [30]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms, utils

In [31]:
img_size = (300, 300)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_transforms = {
    'raw': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ]),
    'prep': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5023, 0.5017, 0.5019], [0.1245, 0.0934, 0.0581])
    ]),
    'vessel': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
    ])}

model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
num_ftrs = model.classifier.in_features
model.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))

Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [32]:
for path in glob.glob('./models/*.pt'):
    
    if path.split('\\')[-1].split('.')[0].split('_')[1] == 'prep': name = '_prep'
    elif path.split('\\')[-1].split('.')[0].split('_')[1] == 'vessel': name = '_vessel'
    else: name = '_raw'

    
    if name == '_raw': data_dir = 'data/data_m'
    else: data_dir = 'data/data_m'+name

    image_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms[name[1:]])
    dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=14, shuffle=True, num_workers=4)
    dataset_size = len(image_dataset)

    model.load_state_dict(torch.load(path))
    model = model.to(device)
    was_training = model.training
    model.eval()
    test_acc = 0
    for i, data in enumerate(dataloader, 0):
        samples, labels = data
        with torch.no_grad():            
            samples, labels = samples.to(device), labels.to(device)
            output = model(samples)
            loss = nn.CrossEntropyLoss()(output, labels)
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            test_acc += torch.sum(correct.float())
            torch.cuda.empty_cache()
            model.train(mode=was_training)
    print('Model: {}, Accuracy: {:.2f} %'.format(path.split('\\')[-1].split('.')[0], 100*test_acc/dataset_size))

Model: densenet121_e50_s300_b14, Accuracy: 84.48 %
Model: densenet121_prep_e50_s300_b14, Accuracy: 91.38 %
Model: densenet121_vessel_e50_s300_b14, Accuracy: 80.00 %
